In [1]:
import pandas as pd
tweet=pd.read_csv("D:/安装包/clean_tweets.txt",sep='\t',names=['date','text','loc','author'])
print(tweet)

                       date  \
0       2020-01-12 12:30:00   
1       2020-01-22 03:00:16   
2       2020-01-24 10:57:46   
3       2020-01-27 06:00:07   
4       2020-01-27 08:00:12   
...                     ...   
258162  2020-12-28 13:15:06   
258163  2020-12-28 14:00:06   
258164  2020-12-28 14:45:02   
258165  2020-12-28 16:15:06   
258166  2020-12-28 16:32:03   

                                                     text    loc author  
0       The diving bell spider or water spider is the ...  CHINA   CCTV  
1       Faced with the fastchanging situation of the n...  CHINA   CCTV  
2       The central China metropolitan of Wuhan will f...  CHINA   CCTV  
3       Medical teams from across China left for Wuhan...  CHINA   CCTV  
4       Many places across China have taken strict con...  CHINA   CCTV  
...                                                   ...    ...    ...  
258162  Stock futures rose after President Trump signe...     US    WSJ  
258163  A citizen journalist in Chi

In [2]:
#coding: utf-8
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from wordcloud import STOPWORDS  #停用词

#补充停用词
STOPWORDS.add('covid')  
STOPWORDS.add('coronavirus')
STOPWORDS.add('covid19')
STOPWORDS.add('corona')
STOPWORDS.add('rt')
STOPWORDS.add('pandemic')
STOPWORDS.add('trump')

STOPWORDS.add('france')  
STOPWORDS.add('china')
STOPWORDS.add('uk')
STOPWORDS.add('us')
STOPWORDS.add('germany')
STOPWORDS.add('brazil')
STOPWORDS.add('spain')

In [3]:
#---------------------  第一步 读取数据(已分词)  ----------------------
corpus = []

# 读取预料 一行预料为一个文档
for index,line in tweet.iterrows():
    cor=str(line['text'])
    corpus.append(cor.strip())

In [4]:
#-----------------------  第二步 计算TF-IDF值  ----------------------- 
# 设置特征数
n_features = 2000

tf_vectorizer = TfidfVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words=set(STOPWORDS),
                                max_df = 0.99,
                                min_df = 0.002) #去除文档内出现几率过大或过小的词汇

tf = tf_vectorizer.fit_transform(corpus)

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
#-------------------------  第三步 LDA分析  ------------------------ 
from sklearn.decomposition import LatentDirichletAllocation

# 设置主题数
n_topics = 4

lda = LatentDirichletAllocation(n_components=n_topics,
                                max_iter=100,
                                learning_method='online',
                                learning_offset=50,
                                random_state=0)
lda.fit(tf)

# 显示主题数 model.topic_word_
print(lda.components_)
# 几个主题就是几行 多少个关键词就是几列 
print(lda.components_.shape)                         

# 计算困惑度
print(u'困惑度：')
print(lda.perplexity(tf,sub_sampling = False))

# 主题-关键词分布
def print_top_words(model, tf_feature_names, n_top_words):
    for topic_idx,topic in enumerate(model.components_):  # lda.component相当于model.topic_word_
        print('Topic #%d:' % topic_idx)
        print(' '.join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]]))
        print("")

# 定义好函数之后 暂定每个主题输出前10个关键词
n_top_words = 10                                       
tf_feature_names = tf_vectorizer.get_feature_names()
# 调用函数
print_top_words(lda, tf_feature_names, n_top_words)

In [ ]:
from pandas.core.frame import DataFrame
res = lda.transform(tf)
res=DataFrame(res)
tweet['class']=res.idxmax(1)
tweet.head(20)

#tweet.to_csv('/content/drive/MyDrive/yuchuli/LDA.txt',sep='\t',header=None,index=None)

In [ ]:
import joblib
#保存模型
model_file = "D:/安装包/1tweet/lda_model4.pk"
joblib.dump(lda,model_file)

In [5]:
#加载训练好的模型
import joblib
model_file = "D:/安装包/LDA/lda_model3.pk"
#词向量
lda = joblib.load(model_file)

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator LatentDirichletAllocation from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
#------------------------  第四步 可视化分析  ------------------------- 
import pandas
import numpy
import pyLDAvis
import pyLDAvis.sklearn

#pyLDAvis.enable_notebook()

data = pyLDAvis.sklearn.prepare(lda,tf,tf_vectorizer)
print(data)

#显示图形
pyLDAvis.show(data,local=False)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.076437 -0.362042       1        1  29.122820
2     -0.083798  0.157872       2        1  25.149057
0     -0.205844  0.155942       3        1  25.034437
1      0.366079  0.048229       4        1  20.693687, topic_info=         Term         Freq        Total Category  logprob  loglift
144     cases  5254.000000  5254.000000  Default  30.0000  30.0000
229    deaths  3721.000000  3721.000000  Default  29.0000  29.0000
532     masks  2710.000000  2710.000000  Default  28.0000  28.0000
500  lockdown  6072.000000  6072.000000  Default  27.0000  27.0000
629  patients  2018.000000  2018.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
577      need   917.724812  1108.926903   Topic4  -4.8827   1.3861
336     first  1444.993993  2965.010303   Topic4  -4.4288   0.8566
840     study   940.965468  13

127.0.0.1 - - [20/May/2021 11:12:10] "GET / HTTP/1.1" 200 -
